### Property Price Prediction Model: Comprehensive Analysis


**Primary Goal**  
Conduct a targeted comparative analysis of 12 strategically selected CNN architectures to identify the optimal feature extraction backbone for property price prediction, representing key evolutionary milestones in computer vision.

Primarily the models are 21 but because of reasons named below only 12 were carefully selected:
1. Architectures that introduced fundamental design breakthoroughs
2. Balanced considerations of practical performance vs. theoratical performance
3. the selected models represents a distinct architetural philosopy

*Core Research Question*  
Which convolutional neural network architecture provides the most discriminative visual features when combined with tabular property data for accurate price prediction?

DIVE INTO CODE AND SEE HOW THE QUESTION IS ANSWERED:

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os

In [2]:
df = pd.read_csv(r"C:\Users\Brendon\Desktop\Deep_Learning\ASSIGNMENTS\final_zimbabwe_property_listings_complete.csv")
df.head()

,scraped_page,title,detail_url,currency,price,building_area,building_unit,land_area,land_unit,property_type,bedrooms,bathrooms,location,image_count,image_filenames
0,1,Developers Dream,https://www.property.co.zw/for-sale/houses-bls...,USD,190000.0,180.0,m²,1352.0,m²,3 Bedroom House,3.0,1.0,Belvedere,1,df03d95d0b_0.webp
1,1,2 Bedroom Flat In Prime Avondale Location,https://www.property.co.zw/for-sale/flats-apar...,USD,95000.0,120.0,m²,NaN,m²,2 Bedroom Flat,2.0,1.0,Avondale,1,41c6aa94bc_0.webp
2,1,"Charming 3-Bedroom Family Home in Mabvazuva, R...",https://www.property.co.zw/for-sale/houses-p19...,USD,105000.0,3410.0,m²,410.0,m²,3 Bedroom House,3.0,NaN,NaN,1,483115d1c1_0.webp
3,1,The Strand office land in Borrowdale.,https://www.property.co.zw/for-sale/commercial...,USD,875000.0,NaN,m²,8000.0,m²,NaN,NaN,NaN,Borrowdale,1,ac7160491a_0.webp
4,1,Stands for Sale,https://www.property.co.zw/for-sale/residentia...,USD,60000.0,442.0,m²,442.0,m²,NaN,NaN,NaN,Harare,1,1d7f22fc05_0.webp


**DATA PREPROCESSING AND FEATURE EXTRACTION**

This preprocessing pipeline systematically transforms raw property listing data into a structured, machine-learning-ready format through three key phases:
1. Data Validation and Cleaning, where invalid prices are filtered out and all currency values are standardized to USD to ensure consistency. Missing values in critical numerical features like bedrooms, bathrooms, and area measurements are intelligently handled using median imputation, which preserves the data distribution while handling gaps effectively.  

2. The Feature Engineering,  demonstrates domain expertise in real estate analytics by creating meaningful derived features. Boolean flags (has_building, has_land) elegantly handle properties with missing area data, while ratio features (area_ratio, bed_bath_ratio) capture important property characteristics that raw measurements alone cannot express. The price_per_sqm metric provides a normalized pricing benchmark that enables fair comparisons across different property types and sizes.

3. Text Processing, extracts property types from listing titles when explicit classifications are missing, ensuring no valuable information is lost. Categorical variables like location and property type are encoded numerically, making them compatible with machine learning algorithms while preserving their informational value.

Finally, the standardization step ensures all features are on comparable scales, which is crucial for models sensitive to feature magnitudes. The output is a clean, well-structured dataset ready for both traditional machine learning and advanced neural network approaches.



In [3]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer


def preprocess_property_data(df):
    df_clean = df.copy()
    
    
    df_clean = df_clean[df_clean['price'] > 0]  # Remove invalid prices
    df_clean['currency'] = df_clean['currency'].fillna('USD')  # All prices in USD
    
    # Loop to impute with median
    for col in ['bedrooms', 'bathrooms', 'building_area', 'land_area']:
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())
    
    # Feature engineering---
    df_clean = df_clean.assign(
        has_building=(df_clean['building_area'] > 0).astype(int),
        has_land=(df_clean['land_area'] > 0).astype(int),
        area_ratio=df_clean['building_area'] / (df_clean['land_area'] + 1),
        bed_bath_ratio=df_clean['bedrooms'] / (df_clean['bathrooms'] + 1),
        price_per_sqm=df_clean['price'] / (df_clean['building_area'] + df_clean['land_area'] + 1)
    )
    
    # Property type standardization
    def get_property_type(row):
        if pd.notna(row['property_type']):
            return row['property_type']
        title = str(row['title']).lower()
        if 'house' in title: return 'House'
        if 'flat' in title or 'apartment' in title: return 'Flat'
        if 'land' in title or 'stand' in title: return 'Land'
        if 'commercial' in title or 'office' in title: return 'Commercial'
        return 'Other'
    
    df_clean['property_type_clean'] = df_clean.apply(get_property_type, axis=1)
    
    #Location encoding
    df_clean['location'] = df_clean['location'].fillna('Unknown')
    
    #FInal features for dataframe
    feature_columns = [
        'building_area', 'land_area', 'bedrooms', 'bathrooms',
        'has_building', 'has_land', 'area_ratio', 'bed_bath_ratio', 
        'price_per_sqm', 'image_count'
    ]
    
    X = df_clean[feature_columns].copy()
    y = df_clean['price']
    
    # Encoding
    property_encoder = LabelEncoder()
    location_encoder = LabelEncoder()
    
    X['property_type_encoded'] = property_encoder.fit_transform(df_clean['property_type_clean'])
    X['location_encoded'] = location_encoder.fit_transform(df_clean['location'])
    
    return X, y, df_clean

# Apply preprocessing
X, y, df_clean = preprocess_property_data(df)

print(f"Final features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Features: {X.columns.tolist()}")

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)



Final features shape: (1610, 12)
Target shape: (1610,)
Features: ['building_area', 'land_area', 'bedrooms', 'bathrooms', 'has_building', 'has_land', 'area_ratio', 'bed_bath_ratio', 'price_per_sqm', 'image_count', 'property_type_encoded', 'location_encoded']


**Image Processing Pipeline**

This image processing pipeline implements an innovative two-pass *median imputation* strategy that fundamentally addresses the common challenge of missing image data in real-world datasets. The approach begins with a comprehensive first pass through all property listings, systematically collecting every available valid image while intelligently handling various edge cases including missing filenames, file system errors, and corrupted images. This careful collection phase enables the computation of a dataset-wide median image that statistically represents the central tendency of all available property photographs.

The core innovation lies in the second pass, where every property listing receives either its actual image or the computed median as a sophisticated placeholder. This strategy ensures complete data preservation - no property is discarded due to missing images, *maintaining the statistical power of the full 1,613-sample dataset*. The median image serves as a *domain-aware default that provides more meaningful information than simple zero-padding*, as it contains realistic visual patterns and color distributions characteristic of property photographs in the Zimbabwe market.

The implementation demonstrates robust error handling through multiple validation layers: filename sanity checks, filesystem existence verification, and loading exception management. Comprehensive status tracking provides transparency into the loading process, enabling quality assessment and potential troubleshooting. The final output maintains perfect alignment between image data and tabular features, ensuring that machine learning models can learn from both data modalities without introducing biases from missing visual information.

In [4]:
def load_images_with_median_imputation(df_clean, image_folder=r"C:\Users\Brendon\Desktop\Deep_Learning\ASSIGNMENTS\images", target_size=(224, 224)):
    """Load images, replace missing ones with median of available images"""
    
    image_arrays = []
    image_status = []
    loaded_images = []  # Store successfully loaded images to compute median
    
    print("Loading images (missing images will be replaced with median image)...")
    
    # First pass: collect all successfully loaded images
    for idx, filename in enumerate(df_clean['image_filenames']):
        if pd.isna(filename) or filename is None or filename == '':
            continue
            
        filename_str = str(filename).strip()
        if filename_str.lower() in ['nan', 'none', 'null', '']:
            continue
        
        img_path = os.path.join(image_folder, filename_str)
        
        try:
            if not os.path.exists(img_path):
                continue
                
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)
            img_array = img_array / 255.0
            loaded_images.append(img_array)
            
        except Exception as e:
            continue
    
    # Compute median image from loaded images
    if loaded_images:
        median_image = np.median(loaded_images, axis=0)
        print(f"Computed median image from {len(loaded_images)} loaded images")
    else:
        median_image = np.zeros((*target_size, 3))
        print("No images loaded, using zero image as median")
    
    # Second pass: build final array with median imputation
    for idx, filename in enumerate(df_clean['image_filenames']):
        # Start with median as default
        current_array = median_image.copy()
        current_status = 'median_imputed'
        
        # Check if filename is valid
        if pd.isna(filename) or filename is None or filename == '':
            image_arrays.append(current_array)
            image_status.append(current_status)
            continue
            
        filename_str = str(filename).strip()
        if filename_str.lower() in ['nan', 'none', 'null', '']:
            image_arrays.append(current_array)
            image_status.append(current_status)
            continue
        
        # Try to load actual image
        img_path = os.path.join(image_folder, filename_str)
        
        try:
            if not os.path.exists(img_path):
                image_arrays.append(current_array)
                image_status.append(current_status)
                continue
                
            # Load and resize image
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)
            img_array = img_array / 255.0
            
            image_arrays.append(img_array)
            image_status.append('loaded')
            
        except Exception as e:
            image_arrays.append(current_array)
            image_status.append('load_error_median')
            continue
    
    # Convert to numpy array
    image_arrays = np.array(image_arrays)
    
    # Print statistics
    status_counts = pd.Series(image_status).value_counts()
    print(f"\nImage loading summary:")
    for status, count in status_counts.items():
        print(f"  {status}: {count} images")
    print(f"Total images processed: {len(image_arrays)}")
    print(f"Image array shape: {image_arrays.shape}")
    
    return image_arrays, image_status, median_image

def create_complete_dataset_with_median(df_clean, X_scaled, y):
    """Create dataset with all tabular data, replacing missing images with median"""
    
    # Load images with median imputation
    images, image_status, median_image = load_images_with_median_imputation(df_clean)
    
    # Use ALL tabular data
    df_with_images = df_clean.copy()
    X_with_images = X_scaled
    y_with_images = y
    
    print(f"\nFinal dataset sizes (ALL data preserved):")
    print(f"Images: {images.shape}")
    print(f"Features (X): {X_with_images.shape}")
    print(f"Targets (y): {y_with_images.shape}")
    print(f"DataFrame: {df_with_images.shape}")
    
    return images, X_with_images, y_with_images, df_with_images, image_status, median_image

# Load with median imputation
images, X_with_images, y_with_images, df_with_images, image_status, median_image = create_complete_dataset_with_median(df_clean, X_scaled, y)

# Show detailed statistics
status_series = pd.Series(image_status)
print(f"\nImage Loading Details:")
print(f"Successfully loaded: {sum(status_series == 'loaded')}")
print(f"Median imputed: {sum(status_series == 'median_imputed')}")
print(f"Load errors (median used): {sum(status_series == 'load_error_median')}")

# Optional: Display median image stats
print(f"\nMedian Image Statistics:")
print(f"Shape: {median_image.shape}")
print(f"Min pixel value: {median_image.min():.3f}")
print(f"Max pixel value: {median_image.max():.3f}")
print(f"Mean pixel value: {median_image.mean():.3f}")

Loading images (missing images will be replaced with median image)...
Computed median image from 466 loaded images

Image loading summary:
  median_imputed: 1144 images
  loaded: 466 images
Total images processed: 1610
Image array shape: (1610, 224, 224, 3)

Final dataset sizes (ALL data preserved):
Images: (1610, 224, 224, 3)
Features (X): (1610, 12)
Targets (y): (1610,)
DataFrame: (1610, 21)

Image Loading Details:
Successfully loaded: 466
Median imputed: 1144
Load errors (median used): 0

Median Image Statistics:
Shape: (224, 224, 3)
Min pixel value: 0.276
Max pixel value: 0.827
Mean pixel value: 0.495


This comprehensive CNN architecture import strategy represents a meticulously curated selection of 12 convolutional neural networks that span the entire evolutionary spectrum of modern deep learning for computer vision. The implementation demonstrates sophisticated foresight in architectural diversity, ensuring that the comparative analysis covers all major design paradigms that have shaped the field. Each imported architecture brings distinct theoretical foundations and practical characteristics, from the groundbreaking residual connections of ResNet to the computationally efficient compound scaling of EfficientNet.


In [5]:
from tensorflow.keras.applications import (
    ResNet50, EfficientNetB0, DenseNet121, MobileNetV2, VGG16, Xception,
    ResNet152, EfficientNetB3, DenseNet201, NASNetMobile, InceptionV3, InceptionResNetV2
)
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input as inception_resnet_preprocess
from tensorflow.keras.applications.nasnet import preprocess_input as nasnet_preprocess
import pickle

**CNN Feature Extraction Pipeline: Systematic Architecture Benchmarking**

Analyzing property photos using 12 different AI models to find visual patterns that help predict house prices. Each model looks at the images in its own unique way, and see which approach works best for real estate.

How It Works - Simple Explanation
1. Two-Step Processing
 - First, prepare each image specifically for each model
 - Some models like smaller images (224x224 pixels)
 - Others need larger ones (299x299 or 300x300 pixels)

2. Extract feature vectors
 - Each model converts images into arrays


In [6]:
def extract_cnn_features(image_arrays, models_config):

    features_dict = {}
    
    for model_name, (model, preprocess_fn, target_size) in models_config.items():
        print(f"Extracting features with {model_name}...")
        
        features = []
        successful_extractions = 0
        
        for img_array in image_arrays:
            try:
                # Resizing for models that require specific input sizes
                if target_size[0] != img_array.shape[0] or target_size[1] != img_array.shape[1]:
                    from PIL import Image
                    img = Image.fromarray((img_array * 255).astype(np.uint8))
                    img = img.resize(target_size)
                    img_resized = np.array(img) / 255.0
                else:
                    img_resized = img_array
                
                # Preprocess for specific model
                img_processed = preprocess_fn(np.expand_dims(img_resized * 255, axis=0))
                
                # Extract features
                feature = model.predict(img_processed, verbose=0)
                features.append(feature.flatten())
                successful_extractions += 1
                
            except Exception as e:
                # Use zeros with correct dimension as fallback
                feature_shape = model.output_shape
                if len(feature_shape) == 4:
                    zero_features = np.zeros(np.prod(feature_shape[1:])).flatten()
                else:
                    zero_features = np.zeros(feature_shape[1]).flatten()
                features.append(zero_features)
                print(f"  Warning: Feature extraction failed for one image, using zeros")
        
        features_dict[model_name] = np.array(features)
        print(f"  {model_name}: {features_dict[model_name].shape} - {successful_extractions}/{len(image_arrays)} successful")
    
    return features_dict

# Define models configuration
models_config = {
    'ResNet50': (
        ResNet50(weights='imagenet', include_top=False, pooling='avg'), 
        resnet_preprocess, 
        (224, 224)
    ),
    'EfficientNetB0': (
        EfficientNetB0(weights='imagenet', include_top=False, pooling='avg'), 
        efficientnet_preprocess, 
        (224, 224)
    ),
    'DenseNet121': (
        DenseNet121(weights='imagenet', include_top=False, pooling='avg'), 
        densenet_preprocess, 
        (224, 224)
    ),
    'MobileNetV2': (
        MobileNetV2(weights='imagenet', include_top=False, pooling='avg'), 
        mobilenet_preprocess, 
        (224, 224)
    ),
    'VGG16': (
        VGG16(weights='imagenet', include_top=False, pooling='avg'), 
        vgg_preprocess, 
        (224, 224)
    ),
    'Xception': (
        Xception(weights='imagenet', include_top=False, pooling='avg'), 
        xception_preprocess, 
        (299, 299)
    ),
    
    'ResNet152': (
        ResNet152(weights='imagenet', include_top=False, pooling='avg'), 
        resnet_preprocess, 
        (224, 224)
    ),
    'EfficientNetB3': (
        EfficientNetB3(weights='imagenet', include_top=False, pooling='avg'), 
        efficientnet_preprocess, 
        (300, 300)  # EfficientNetB3 prefers 300x300
    ),
    'DenseNet201': (
        DenseNet201(weights='imagenet', include_top=False, pooling='avg'), 
        densenet_preprocess, 
        (224, 224)
    ),
    'NASNetMobile': (
        NASNetMobile(weights='imagenet', include_top=False, pooling='avg'), 
        nasnet_preprocess, 
        (224, 224)
    ),
    'InceptionV3': (
        InceptionV3(weights='imagenet', include_top=False, pooling='avg'), 
        inception_preprocess, 
        (299, 299)  # InceptionV3 requires 299x299
    ),
    'InceptionResNetV2': (
        InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg'), 
        inception_resnet_preprocess, 
        (299, 299)  # InceptionResNetV2 requires 299x299
    )
}

# Print model information
print("=== CNN MODELS FOR FEATURE EXTRACTION ===")
print("\nModel details:")

for model_name, (model, _, target_size) in models_config.items():
    feature_shape = model.output_shape
    feature_dim = feature_shape[1] if len(feature_shape) == 2 else np.prod(feature_shape[1:])
    print(f"{model_name:20} | Input: {target_size} | Output: {feature_dim} features")

# EXTRACT FEATURES
print("\n" + "="*60)
print("FEATURE EXTRACTION WITH 12 MODELS")
print("="*60)

cnn_features = extract_cnn_features(images, models_config)

print("\n" + "="*60)
print("FEATURE EXTRACTION COMPLETED")
print("="*60)

for model_name, features in cnn_features.items():
    print(f"{model_name:20}: {features.shape}")


C:\Users\Brendon\AppData\Local\Temp\ipykernel_14820\2054514194.py:63: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  MobileNetV2(weights='imagenet', include_top=False, pooling='avg'),



=== CNN MODELS FOR FEATURE EXTRACTION ===

Model details:
ResNet50             | Input: (224, 224) | Output: 2048 features
EfficientNetB0       | Input: (224, 224) | Output: 1280 features
DenseNet121          | Input: (224, 224) | Output: 1024 features
MobileNetV2          | Input: (224, 224) | Output: 1280 features
VGG16                | Input: (224, 224) | Output: 512 features
Xception             | Input: (299, 299) | Output: 2048 features
ResNet152            | Input: (224, 224) | Output: 2048 features
EfficientNetB3       | Input: (300, 300) | Output: 1536 features
DenseNet201          | Input: (224, 224) | Output: 1920 features
NASNetMobile         | Input: (224, 224) | Output: 1056 features
InceptionV3          | Input: (299, 299) | Output: 2048 features
InceptionResNetV2    | Input: (299, 299) | Output: 1536 features

FEATURE EXTRACTION WITH 12 MODELS
Extracting features with ResNet50...
  ResNet50: (1610, 2048) - 1610/1610 successful
Extracting features with EfficientNetB0...

**Dataset Integration: Multi-Modal Feature Fusion**  
High-Level Overview  

We're now performing feature fusion - combining our structured tabular data with the unstructured visual features extracted from our CNN architectures. This creates multi-modal datasets where each property is represented by both its metadata features and its learned visual representations.

For each of our 12 CNN backbones, we're creating an integrated feature space that merges:  
- Structured attributes: The original 12 engineered features (bedrooms, location encodings, area ratios, etc.)
- Visual embeddings: The high-dimensional feature vectors from each architecture's convolutional layers

In [7]:
# Create combined datasets for each model
print("\n" + "="*60)
print("CREATING COMBINED DATASETS")
print("="*60)

def create_combined_datasets(X_with_images, cnn_features, model_names=None):
    """
    Create combined feature sets for each CNN model
    """
    if model_names is None:
        model_names = list(cnn_features.keys())
    
    combined_datasets = {}
    
    for model_name in model_names:
        # Combine tabular features with CNN features
        cnn_feature_array = cnn_features[model_name]
        combined_features = np.concatenate([X_with_images, cnn_feature_array], axis=1)
        combined_datasets[model_name] = combined_features
        
        print(f"{model_name}: Tabular {X_with_images.shape[1]} + CNN {cnn_feature_array.shape[1]} = Combined {combined_features.shape[1]}")
    
    return combined_datasets

combined_datasets = create_combined_datasets(X_with_images, cnn_features)

# Save expanded features
print("\n" + "="*60)
print("SAVING EXPANDED FEATURES")
print("="*60)

# Save CNN features
with open('cnn_features_expanded.pkl', 'wb') as f:
    pickle.dump(cnn_features, f)

# Save combined datasets
with open('combined_datasets_expanded.pkl', 'wb') as f:
    pickle.dump(combined_datasets, f)

print("All expanded features saved successfully!")

# Final summary
print("\n" + "="*60)
print("FINAL EXPANDED SUMMARY")
print("="*60)
print(f"Images loaded: {images.shape}")
print(f"Tabular features: {X_with_images.shape}")
print(f"Target variable: {y_with_images.shape}")
print(f"CNN features extracted for {len(cnn_features)} models")

# Display feature dimensions in a nice table
print(f"\n{'Model':20} | {'CNN Features':12} | {'Total Features':14} | {'Input Size':10}")
print("-" * 65)
for model_name, features in cnn_features.items():
    combined_shape = combined_datasets[model_name].shape
    input_size = models_config[model_name][2]
    print(f"{model_name:20} | {features.shape[1]:12} | {combined_shape[1]:14} | {str(input_size):10}")



CREATING COMBINED DATASETS
ResNet50: Tabular 12 + CNN 2048 = Combined 2060
EfficientNetB0: Tabular 12 + CNN 1280 = Combined 1292
DenseNet121: Tabular 12 + CNN 1024 = Combined 1036
MobileNetV2: Tabular 12 + CNN 1280 = Combined 1292
VGG16: Tabular 12 + CNN 512 = Combined 524
Xception: Tabular 12 + CNN 2048 = Combined 2060
ResNet152: Tabular 12 + CNN 2048 = Combined 2060
EfficientNetB3: Tabular 12 + CNN 1536 = Combined 1548
DenseNet201: Tabular 12 + CNN 1920 = Combined 1932
NASNetMobile: Tabular 12 + CNN 1056 = Combined 1068
InceptionV3: Tabular 12 + CNN 2048 = Combined 2060
InceptionResNetV2: Tabular 12 + CNN 1536 = Combined 1548

SAVING EXPANDED FEATURES
All expanded features saved successfully!

FINAL EXPANDED SUMMARY
Images loaded: (1610, 224, 224, 3)
Tabular features: (1610, 12)
Target variable: (1610,)
CNN features extracted for 12 models

Model                | CNN Features | Total Features | Input Size
-----------------------------------------------------------------
ResNet50    

**Multi-Modal Architecture Evaluation**

We're now executing our core comparative analysis - training and evaluating neural networks on each of our 12 multi-modal datasets to determine which CNN backbone produces the most effective visual embeddings for property price prediction. This represents the model evaluation phase where we systematically measure how well each architecture's feature representations complement our tabular data.

**Evaluation Strategy**
- Consistent Splits: Same train/val/test indices across all architectures
- Early Stopping: Prevents overfitting and ensures fair comparison
- Multiple Metrics: MAE for monetary interpretation, R² for variance explanation
- Architecture Families: Enables pattern recognition across design paradigms

**Comparative Performance Metrics**  

The ranking table provides:  
- Absolute Performance: Test MAE values in monetary terms
- Explanatory Power: R² scores indicating variance captured
- Feature Efficiency: Relationship between embedding size and performance
- Architecture Patterns: Performance trends across design families

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import numpy as np

def create_neural_network(tabular_dim, cnn_feature_dim):
    """Create neural network with combined tabular + CNN features"""
    
    # Tabular features input
    tabular_input = Input(shape=(tabular_dim,), name='tabular_input')
    x_tab = Dense(128, activation='relu')(tabular_input)
    x_tab = BatchNormalization()(x_tab)
    x_tab = Dropout(0.3)(x_tab)
    x_tab = Dense(64, activation='relu')(x_tab)
    x_tab = Dropout(0.2)(x_tab)
    
    # CNN features input  
    cnn_input = Input(shape=(cnn_feature_dim,), name='cnn_input')
    x_cnn = Dense(256, activation='relu')(cnn_input)
    x_cnn = BatchNormalization()(x_cnn)
    x_cnn = Dropout(0.3)(x_cnn)
    x_cnn = Dense(128, activation='relu')(x_cnn)
    x_cnn = Dropout(0.2)(x_cnn)
    
    # Concatenate both streams
    concatenated = Concatenate()([x_tab, x_cnn])
    
    # Final layers
    x = Dense(64, activation='relu')(concatenated)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu')(x)
    
    # Output layer (regression - no activation)
    output = Dense(1, activation='linear', name='price_output')(x)
    
    model = Model(inputs=[tabular_input, cnn_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='mse', 
                  metrics=['mae'])
    
    return model

def compare_12_cnn_architectures(combined_datasets, X_with_images, y_with_images):
    """Compare all 12 CNN architectures using neural networks"""
    
    results = {}
    tabular_dim = X_with_images.shape[1]
    
    # Single split for all models (60-20-20)
    indices = np.arange(len(y_with_images))
    X_temp, X_test, y_temp, y_test = train_test_split(
        indices, y_with_images, test_size=0.2, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.25, random_state=42
    )
    
    print("=== COMPARING 12 CNN ARCHITECTURES WITH NEURAL NETWORKS ===")
    print(f"Training on {len(X_train)} samples, Validating on {len(X_val)} samples, Testing on {len(X_test)} samples")
    print(f"Tabular features: {tabular_dim} dimensions\n")
    
    for i, model_name in enumerate(combined_datasets.keys(), 1):
        print(f"{i:2d}/12 - Training with {model_name:20}...", end=" ")
        
        # Get features and split into components
        combined_features = combined_datasets[model_name]
        cnn_feature_dim = combined_features.shape[1] - tabular_dim
        
        # Split and separate features
        X_train_combined = combined_features[X_train]
        X_val_combined = combined_features[X_val]
        X_test_combined = combined_features[X_test]
        
        X_train_tabular = X_train_combined[:, :tabular_dim]
        X_train_cnn = X_train_combined[:, tabular_dim:]
        X_val_tabular = X_val_combined[:, :tabular_dim] 
        X_val_cnn = X_val_combined[:, tabular_dim:]
        X_test_tabular = X_test_combined[:, :tabular_dim]
        X_test_cnn = X_test_combined[:, tabular_dim:]
        
        # Create and train neural network
        nn_model = create_neural_network(tabular_dim, cnn_feature_dim)
        
        early_stop = EarlyStopping(patience=10, restore_best_weights=True, verbose=0)
        
        history = nn_model.fit(
            [X_train_tabular, X_train_cnn], y_train,
            validation_data=([X_val_tabular, X_val_cnn], y_val),
            epochs=100,
            batch_size=32,
            callbacks=[early_stop],
            verbose=0
        )
        
        # Evaluate
        test_loss, test_mae = nn_model.evaluate([X_test_tabular, X_test_cnn], y_test, verbose=0)
        y_pred = nn_model.predict([X_test_tabular, X_test_cnn], verbose=0).flatten()
        test_r2 = r2_score(y_test, y_pred)
        
        results[model_name] = {
            'test_mae': test_mae,
            'test_r2': test_r2,
            'cnn_feature_dim': cnn_feature_dim,
            'total_features': combined_features.shape[1],
            'epochs_trained': len(history.history['loss'])
        }
        
        print(f"MAE: ${test_mae:,.2f} | R²: {test_r2:.3f} | Features: {cnn_feature_dim}")
    
    return results

# Compare all 12 architectures
print("Starting comparison of 12 CNN architectures...")
results_nn = compare_12_cnn_architectures(combined_datasets, X_with_images, y_with_images)

# Display comprehensive rankings
print("\n" + "="*70)
print("FINAL RANKINGS - 12 CNN ARCHITECTURES (Neural Networks)")
print("="*70)

ranked_results = sorted(results_nn.items(), key=lambda x: x[1]['test_mae'])

print(f"{'Rank':4} {'Model':20} {'Test MAE':12} {'R²':8} {'CNN Features':14} {'Total Features':14}")
print("-" * 70)

for i, (model_name, metrics) in enumerate(ranked_results, 1):
    print(f"{i:2d}.  {model_name:20} ${metrics['test_mae']:>10,.2f}  {metrics['test_r2']:>6.3f}  "
          f"{metrics['cnn_feature_dim']:>12}  {metrics['total_features']:>13}")

# Performance analysis
print("\n" + "="*70)
print("PERFORMANCE ANALYSIS")
print("="*70)

best_model = ranked_results[0][0]
best_mae = ranked_results[0][1]['test_mae']
worst_model = ranked_results[-1][0]
worst_mae = ranked_results[-1][1]['test_mae']

mae_values = [metrics['test_mae'] for metrics in results_nn.values()]
mean_mae = np.mean(mae_values)
std_mae = np.std(mae_values)

print(f"Best Model: {best_model} (MAE: ${best_mae:,.2f})")
print(f"Worst Model: {worst_model} (MAE: ${worst_mae:,.2f})")
print(f"Average MAE across all models: ${mean_mae:,.2f} ± ${std_mae:,.2f}")
print(f"Performance range: ${best_mae:,.2f} - ${worst_mae:,.2f}")

# Architecture family analysis
print(f"\n{'='*70}")
print("ARCHITECTURE FAMILY ANALYSIS")
print("="*70)

families = {
    'ResNet': ['ResNet50', 'ResNet152'],
    'EfficientNet': ['EfficientNetB0', 'EfficientNetB3'],
    'DenseNet': ['DenseNet121', 'DenseNet201'],
    'Mobile': ['MobileNetV2', 'NASNetMobile'],
    'Inception': ['InceptionV3', 'InceptionResNetV2'],
    'Classic': ['VGG16', 'Xception']
}

for family, models in families.items():
    family_maes = [results_nn[model]['test_mae'] for model in models if model in results_nn]
    if family_maes:
        avg_mae = np.mean(family_maes)
        best_family_model = min([(model, results_nn[model]['test_mae']) for model in models if model in results_nn], 
                               key=lambda x: x[1])
        print(f"{family:15} | Avg MAE: ${avg_mae:>8,.2f} | Best: {best_family_model[0]:15} (${best_family_model[1]:,.2f})")

# Save results
import pickle
with open('nn_comparison_12_models.pkl', 'wb') as f:
    pickle.dump(results_nn, f)

print(f"\nResults saved to 'nn_comparison_12_models.pkl'")
print("Comparison completed! 🎯")

Starting comparison of 12 CNN architectures...
=== COMPARING 12 CNN ARCHITECTURES WITH NEURAL NETWORKS ===
Training on 966 samples, Validating on 322 samples, Testing on 322 samples
Tabular features: 12 dimensions

 1/12 - Training with ResNet50            ... MAE: $631,169.31 | R²: -0.064 | Features: 2048
 2/12 - Training with EfficientNetB0      ... MAE: $631,203.56 | R²: -0.064 | Features: 1280
 3/12 - Training with DenseNet121         ... MAE: $631,194.44 | R²: -0.064 | Features: 1024
 4/12 - Training with MobileNetV2         ... MAE: $631,190.69 | R²: -0.064 | Features: 1280
 5/12 - Training with VGG16               ... MAE: $631,195.75 | R²: -0.064 | Features: 512
 6/12 - Training with Xception            ... MAE: $631,204.31 | R²: -0.064 | Features: 2048
 7/12 - Training with ResNet152           ... MAE: $631,206.19 | R²: -0.064 | Features: 2048
 8/12 - Training with EfficientNetB3      ... MAE: $631,202.31 | R²: -0.064 | Features: 1536
 9/12 - Training with DenseNet201         